In [ ]:
import torch
print(torch.__version__)
print("cuda available:", torch.cuda.is_available())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

print("numpy:", np.__version__)
print("torch:", torch.__version__)

In [ ]:
def make_vibration(sample_rate=1000, duration=1.0):
    """
    生成一条合成震动波形（长度 T = sample_rate * duration）
    组合：正弦 + 衰减包络 + 可选脉冲 + 少量噪声
    """
    T = int(sample_rate * duration)
    t = np.linspace(0, duration, T, endpoint=False)

    # 随机参数
    freq = np.random.uniform(80, 250)          # Hz
    amp = np.random.uniform(0.1, 1.0)          # amplitude
    decay = np.random.uniform(1.5, 8.0)        # 衰减速度
    
    # 基础正弦 + 衰减包络
    envelope = np.exp(-decay * t)
    vib = amp * envelope * np.sin(2 * np.pi * freq * t)

    # 可选：加入 0~3 个脉冲（模拟 tap）
    num_pulses = np.random.randint(0, 4)
    for _ in range(num_pulses):
        start = np.random.randint(0, T - 60)
        width = np.random.randint(20, 80)
        pulse_amp = np.random.uniform(0.2, 1.0)
        vib[start:start+width] += pulse_amp

    # 加一点噪声（更像真实）
    vib += np.random.normal(0, 0.02, size=T)

    # 归一化到 [-1, 1] 附近（训练更稳定）
    max_abs = np.max(np.abs(vib)) + 1e-8
    vib = vib / max_abs
    
    return vib.astype(np.float32)

# 生成数据集，5000条，每条一秒，采样率1000Hz
N = 5000
sample_rate = 1000
duration = 1.0
T = int(sample_rate * duration)

X = np.stack([make_vibration(sample_rate, duration) for _ in range(N)], axis=0)  # (N, T)
print("Dataset shape:", X.shape)

# 看几条例子
plt.figure(figsize=(10, 3))
for i in range(3):
    plt.plot(X[i], label=f"sample {i}")
plt.title("Synthetic vibrotactile waveforms (examples)")
plt.legend()
plt.show()

In [ ]:
# 切分 train/val
perm = np.random.permutation(N)
train_idx = perm[:int(0.8 * N)]
val_idx = perm[int(0.8 * N):]

X_train = torch.from_numpy(X[train_idx])
X_val = torch.from_numpy(X[val_idx])

train_loader = DataLoader(TensorDataset(X_train), batch_size=128, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val), batch_size=128, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
class SimpleAE(nn.Module):
    def __init__(self, input_dim=1000, latent_dim=16):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
            nn.Tanh()  # 输出约束在[-1,1]
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat, z

latent_dim = 16
model = SimpleAE(input_dim=T, latent_dim=latent_dim).to(device)
print(model)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def run_epoch(loader, train=True):
    model.train(train)
    total_loss = 0.0
    count = 0
    for (x,) in loader:
        x = x.to(device)
        if train:
            optimizer.zero_grad()
        x_hat, _ = model(x)
        loss = criterion(x_hat, x)
        if train:
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * x.size(0)
        count += x.size(0)
    return total_loss / count

train_losses, val_losses = [], []
epochs = 30

for epoch in range(1, epochs + 1):
    tr = run_epoch(train_loader, train=True)
    va = run_epoch(val_loader, train=False)
    train_losses.append(tr)
    val_losses.append(va)
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} | train loss: {tr:.6f} | val loss: {va:.6f}")

# 画 loss 曲线
plt.figure(figsize=(8, 3))
plt.plot(train_losses, label="train loss")
plt.plot(val_losses, label="val loss")
plt.title("AE Loss (MSE) over epochs")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    # 从验证集取几条
    x = X_val[:5].to(device)
    x_hat, z = model(x)
    x = x.cpu().numpy()
    x_hat = x_hat.cpu().numpy()
    z = z.cpu().numpy()

# 画原始 vs 重建
num_show = 3
plt.figure(figsize=(12, 6))
for i in range(num_show):
    plt.subplot(num_show, 1, i+1)
    plt.plot(x[i], label="original")
    plt.plot(x_hat[i], label="reconstructed", alpha=0.8)
    plt.title(f"Sample {i} | latent z shape: {z[i].shape}")
    plt.legend()
plt.tight_layout()
plt.show()

print("Example latent z (first sample):", z[0])